<a href="https://colab.research.google.com/github/AyushV14/Ai-Content-Generator/blob/main/Toxic_Classifier_Simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hate Speech Detection


In [2]:
!pip install sentence-transformers
!pip install scikit-learn


In [3]:
from google.colab import files
uploaded = files.upload()

import pandas as pd

df = pd.read_csv(list(uploaded.keys())[0])
df.head()

Saving HateSpeechData.csv to HateSpeechData.csv


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [4]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))

def clean_text(t):
    t = str(t).lower()
    t = re.sub(r'[^a-zA-Z\s]', '', t)
    words = [w for w in t.split() if w not in stopwords_set]
    return " ".join(words)

df["clean_text"] = df["tweet"].apply(clean_text)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,clean_text
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...,rt mayasolovely woman shouldnt complain cleani...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,rt mleew boy dats coldtyga dwn bad cuffin dat ...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,rt urkindofbrand dawg rt sbabylife ever fuck b...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,rt cganderson vivabased look like tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,rt shenikaroberts shit hear might true might f...


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"], df["class"], test_size=0.2, random_state=42
)

tfidf = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

model = LinearSVC()
model.fit(X_train_tfidf, y_train)

preds = model.predict(X_test_tfidf)

print("✅ Accuracy:", accuracy_score(y_test, preds))
print(classification_report(y_test, preds))


✅ Accuracy: 0.890054468428485
              precision    recall  f1-score   support

           0       0.49      0.25      0.33       290
           1       0.92      0.95      0.94      3832
           2       0.83      0.82      0.82       835

    accuracy                           0.89      4957
   macro avg       0.74      0.68      0.70      4957
weighted avg       0.88      0.89      0.88      4957



In [6]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression

sbert = SentenceTransformer('all-MiniLM-L6-v2')

train_emb = sbert.encode(list(X_train), show_progress_bar=True)
test_emb = sbert.encode(list(X_test), show_progress_bar=True)

sbert_model = LogisticRegression(max_iter=200)
sbert_model.fit(train_emb, y_train)

sbert_preds = sbert_model.predict(test_emb)

print("✅ SBERT Accuracy:", accuracy_score(y_test, sbert_preds))
print(classification_report(y_test, sbert_preds))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/620 [00:00<?, ?it/s]

Batches:   0%|          | 0/155 [00:00<?, ?it/s]

✅ SBERT Accuracy: 0.8721000605204761
              precision    recall  f1-score   support

           0       0.43      0.16      0.23       290
           1       0.90      0.95      0.93      3832
           2       0.79      0.75      0.77       835

    accuracy                           0.87      4957
   macro avg       0.71      0.62      0.64      4957
weighted avg       0.85      0.87      0.86      4957



In [8]:
def predict_toxicity(text):
    cleaned = clean_text(text)
    tfidf_vector = tfidf.transform([cleaned])
    pred = model.predict(tfidf_vector)[0]

    labels = {
        0: "Hate Speech",
        1: "Offensive Language",
        2: "Neither (Clean)"
    }

    return labels[pred]


In [10]:
import ipywidgets as widgets
from IPython.display import display

text_box = widgets.Textarea(
    value='',
    placeholder='Type a comment here...',
    description='Input:',
    disabled=False,
    layout=widgets.Layout(width='600px', height='100px')
)

button = widgets.Button(
    description='Predict',
    button_style='success',
    tooltip='Click to classify'
)

out = widgets.Output()

def on_button_clicked(b):
    with out:
        out.clear_output()
        prediction = predict_toxicity(text_box.value)
        print("Prediction:", prediction)

button.on_click(on_button_clicked)

display(text_box, button, out)


Textarea(value='', description='Input:', layout=Layout(height='100px', width='600px'), placeholder='Type a com…

Button(button_style='success', description='Predict', style=ButtonStyle(), tooltip='Click to classify')

Output()